This project focuses on building a facial verification app using TensorFlow and Kivy

In [4]:
!pip install tensorflow-gpu==2.8.0
!pip install opencv-python matplotlib


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Check if tf is installed

In [5]:
import tensorflow as tf

print(tf.__version__)
print(tf.test.gpu_device_name())


2.8.0

